In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### 1. manipulate object image

In [2]:
img = cv2.imread('source-data/dt.png', cv2.IMREAD_UNCHANGED)
rows,cols,ch = img.shape

SMALL_RECT_LEFT = rows*2/6
SMALL_RECT_RIGHT = rows*4/6
SMALL_RECT_TOP = cols*2/6
SMALL_RECT_BOTTOM = cols*4/6
small_rectangle_border = ((SMALL_RECT_LEFT, SMALL_RECT_TOP),
                         (SMALL_RECT_RIGHT, SMALL_RECT_TOP),
                         (SMALL_RECT_LEFT, SMALL_RECT_BOTTOM),
                         (SMALL_RECT_RIGHT, SMALL_RECT_BOTTOM))

In [ ]:




pts1 = np.float32([[0,0],[cols,0],[0,rows],[cols, rows]])
pts2 = np.float32([[0,0],[cols-10,20],[20,rows-10],[cols,rows]])

M = cv2.getPerspectiveTransform(pts1,pts2)

# cv2.warpPerspective( img, getPerspectiveTransform, (cols, rows))
dst = cv2.warpPerspective(img,M,(cols+100,rows+100))

imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
dstRGB = cv2.cvtColor(dst, cv2.COLOR_BGR2RGB)
plt.subplot(121),plt.imshow(imgRGB),plt.title('Input')
plt.subplot(122),plt.imshow(dstRGB),plt.title('Output')
plt.show()
dst.shape